Install Required Libraries 


In [ ]:
!pip install tensorflow==1.13.1

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive


In [4]:
import numpy as np

Making test data batch size equal to that required

In [43]:
X_test = np.load('/content/gdrive/MyDrive/ML Datasets/embeddings_custom_2.npy')
test_len = len(X_test)

In [44]:
X_test.shape

(500, 256)

In [45]:
val = int(np.ceil((len(X_test)/32))*32 - len(X_test))
X_test = np.append(X_test,X_test[len(X_test)-val:len(X_test)],axis = 0)

In [46]:
np.save('/content/gdrive/MyDrive/ML Datasets/test_1.npy',X_test)

In [29]:
#misc
Y_test = np.load('/content/gdrive/MyDrive/ML Datasets/test_1_y.npy')
val = len(X_test) - len(Y_test)
Y_test = np.append(Y_test,Y_test[len(Y_test)-val:len(Y_test)],axis = 0)
np.save('/content/gdrive/MyDrive/ML Datasets/test_1_y.npy',Y_test)


Running GANMM to get the 1.Sampled data 2. Clusters

In [ ]:
!python "/content/gdrive/MyDrive/ML Datasets/GANMM-master/main.py" sae_mnist

Augmenting fake data to balance out clusters

In [49]:
fake1 = np.load('/content/gdrive/MyDrive/ML Datasets/fakeData.npy')

In [50]:
fake1.shape

(4, 352, 256)

In [51]:
Y_pred = np.load('/content/gdrive/MyDrive/ML Datasets/GANMM-master/GanmmPred.npy')

In [52]:
Y_pred.shape

(512,)

Deciding How much to add data

In [53]:
(unique, counts) = np.unique(Y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[  0, 186],
       [  1, 142],
       [  2, 108],
       [  3,  76]])

In [54]:
add_num = [0,0,0,0]
avg = int(len(X_test)/4 )
for i in range(0,4):
  if(avg-frequencies[i][1] > 0):
    add_num[i] = min(max(avg-frequencies[i][1],0),80)
  

In [ ]:
len(X_test)

Finally adding this to X_test and saving

In [ ]:
add_num

In [57]:
for i in range(0,4):
  if(add_num[i]>0):
    X_test = np.append(X_test,fake1[i][:add_num[i]],axis = 0)

Using Spectral Clustering On this data(X_test)

In [ ]:
X_test.shape

In [41]:
#After this step we run spectral clustering in balanced dataset

In [ ]:
!pip install spectralcluster
from spectralcluster import SpectralClusterer

In [61]:
clusterer = SpectralClusterer(
                  min_clusters=1,
                  max_clusters=10,
                  p_percentile=0.95,
                  gaussian_blur_sigma=1)
curr_labels = clusterer.predict(X_test)
np.save('/content/gdrive/MyDrive/ML Datasets/AugmentedCluster_custom.npy',curr_labels[:test_len])